# Homework 2

Submit your *.ipynb through Gradescope by downloading: `File` ⇒ `Download` ⇒ `Download .ipynb`, and then submit with your PDF via link to your repository.

### Setup

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=57b269076843f9eb5c97278b5b505efd79910575fccfa80230ec952d6aec5a36
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [2]:
#@title Import PySpark and create SparkContext

import itertools
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
#@title Download the data from the course website
!wget https://course.ccs.neu.edu/cs6220/fall2023/homework-3/soc-LiveJournal1Adj.txt

--2023-10-10 17:34:58--  https://course.ccs.neu.edu/cs6220/fall2023/homework-3/soc-LiveJournal1Adj.txt
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156181 (4.0M) [text/plain]
Saving to: ‘soc-LiveJournal1Adj.txt’

soc-LiveJournal1Adj 100%[===================>]   3.96M  --.-KB/s    in 0.1s    

2023-10-10 17:34:58 (32.3 MB/s) - ‘soc-LiveJournal1Adj.txt’ saved [4156181/4156181]



### Load the data in!

In [4]:
# Read the data in
lines = sc.textFile("soc-LiveJournal1Adj.txt", 1)
lines = lines.map(lambda line: line.split())

In [5]:
lines.take(2)

[['0',
  '1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'],
 ['1',
  '0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592']]

Using customized function to flatMap the lines to get tuples of user pairs with mutual friends

In [6]:
def mapFriend(user_friends):
  res =[]
  if len(user_friends) == 2:
    user, friends = user_friends
    friends = friends.split(',')
    for i in range(len(friends) - 1):
      for j in range(i + 1, len(friends)):
        res.append(((friends[i], friends[j]), 1))
  return res
aggregatedPairs = lines.flatMap(mapFriend)
# aggregatedPairs.filter(lambda x: x[0] not in directFriendSet)

Using flatMap with customized function to get set of tuples of pairs of users that are direct friend with each other

In [7]:
def findDirectFriend(user_friends):
  res = set()
  if len(user_friends) == 2:
    user, friends = user_friends
    friends = friends.split(',')
    for i in range(len(friends)):
      res.add((user, friends[i]))
  return res
directFriendSet = set(lines.flatMap(findDirectFriend).collect())
print(list(directFriendSet)[:10])

[('6570', '37830'), ('27225', '27224'), ('36014', '37856'), ('37044', '36903'), ('1421', '6939'), ('48964', '18348'), ('34373', '23595'), ('24322', '25268'), ('23581', '34419'), ('9287', '9252')]


Filter out friends that are direct friends from the aggregatedPairs

In [8]:
filteredAggregatedPair = aggregatedPairs.filter(lambda x: x[0] not in directFriendSet)
filteredAggregatedPair.take(5)

[(('1', '2'), 1),
 (('1', '3'), 1),
 (('1', '4'), 1),
 (('1', '6'), 1),
 (('1', '7'), 1)]

Reduce the filtered AggregatedPair by key to add up counts of each tuple, sort it by the key.

In [9]:
mutualMap = filteredAggregatedPair.reduceByKey(lambda a, b: a + b).sortByKey()
mappedMutualMap = mutualMap.map(lambda x: (x[0][0], [[x[0][1]], [x[1]]]))
def aggregateValues(a, b):
  return [a[0] + b[0], a[1] + b[1]]
aggregateMutualMap = mappedMutualMap.reduceByKey(aggregateValues)

Generate recommendations from the data processed in the previous step using map with customized function

In [10]:
def generateRecommendation(x):
  zipped = list(zip(x[1][0], x[1][1]))
  sortedList = sorted(zipped, key=lambda t: (-int(t[1]), int(t[0])))
  topTenzipp = sortedList[:10]
  topTen1, topTen2 = zip(*topTenzipp)
  return (x[0], topTen1)
recommendations = aggregateMutualMap.map(generateRecommendation)
recFor11 = recommendations.filter(lambda x: x[0] == '11').collect()
recFor924 = recommendations.filter(lambda x: x[0] == '924').collect()
recFor8941 = recommendations.filter(lambda x: x[0] == '8941').collect()
recFor8942 = recommendations.filter(lambda x: x[0] == '8942').collect()
recFor9019 = recommendations.filter(lambda x: x[0] == '9019').collect()
recFor9020 = recommendations.filter(lambda x: x[0] == '9020').collect()
recFor9021 = recommendations.filter(lambda x: x[0] == '9021').collect()
recFor9022 = recommendations.filter(lambda x: x[0] == '9022').collect()
recFor9990 = recommendations.filter(lambda x: x[0] == '9990').collect()
recFor9992 = recommendations.filter(lambda x: x[0] == '9992').collect()
recFor9993 = recommendations.filter(lambda x: x[0] == '9993').collect()
print(recFor11)
print(recFor924)
print(recFor8941)
print(recFor8942)
print(recFor9019)
print(recFor9020)
print(recFor9021)
print(recFor9022)
print(recFor9990)
print(recFor9992)
print(recFor9993)

[('11', ('27552', '7785', '27573', '27574', '27589', '27590', '27600', '27617', '27620', '27667'))]
[('924', ('2409', '6995', '11860', '15416', '43748', '45881'))]
[('8941', ('8943', '8944'))]
[('8942', ('8943', '8944'))]
[('9019', ('9022', '9023'))]
[('9020', ('9021', '9022', '9023'))]
[('9021', ('9022', '9023'))]
[('9022', ('9023',))]
[('9990', ('13134', '13478', '13877', '34299', '34485', '34642', '37941'))]
[('9992', ('35667', '9989', '9991'))]
[('9993', ('13134', '13478', '13877', '34299', '34485', '34642', '37941'))]


Output the result to output.txt

In [11]:
import json
result = recommendations.collect()
with open("output.txt", "w") as file:
  json.dump(result, file)